<a href="https://colab.research.google.com/github/LeonVillanueva/CoLab/blob/master/Goolge_CoLab_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 64kB/s 
     |████████████████████████████████| 3.1MB 50.3MB/s 
     |████████████████████████████████| 501kB 27.9MB/s 


In [0]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [0]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, Concatenate, GlobalMaxPooling2D, MaxPooling1D, GaussianNoise, BatchNormalization, MaxPooling2D, SimpleRNN, GRU, LSTM, GlobalMaxPooling1D, Embedding, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import mnist

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
from scipy import stats
import math
import seaborn as sns
import re
from nltk.stem import WordNetLemmatizer
import re

### Loading the Data

In [0]:
mnist = mnist

In [8]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [9]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [12]:
np.max (X_train[0])

255

In [0]:
X_train = X_train / 255.0 * 2 + 1
X_test = X_test / 255.0 * 2 + 1

In [0]:
X_train = X_train.reshape (-1, 28**2)

In [0]:
X_test = X_test.reshape (-1, 28**2)

In [0]:
latent = 128

In [0]:
def generator_nn (latent):
  i = Input (shape=(latent,))
  x = Dense (256, activation=LeakyReLU(alpha=0.2))  (i)
  x = BatchNormalization (momentum=0.8) (x)
  x = Dense (512, activation=LeakyReLU(alpha=0.2))  (x)
  x = BatchNormalization (momentum=0.8) (x)
  x = Dense (1024, activation=LeakyReLU(alpha=0.2))  (x)
  x = BatchNormalization (momentum=0.8) (x)
  x = Dense (28**2, activation='tanh') (x)

  model = Model (i, x)
  return model

In [0]:
def discriminator_nn (image):
  i = Input (shape=(image,))
  x = Dense (512, activation=LeakyReLU(alpha=0.2))  (i)
  x = Dense (256, activation=LeakyReLU(alpha=0.2))  (x)
  x = Dense (1, activation='sigmoid') (x)
  model = Model (i, x)
  return model

In [0]:
discriminator = discriminator_nn (28**2)
discriminator.compile (
    loss = 'binary_crossentropy',
    optimizer = Adam (0.0002, 0.5),
    metrics=['accuracy']
)

In [0]:
generator = generator_nn (latent)
z = Input (shape=(latent,))
image = generator (z)

In [0]:
discriminator.trainable = False

In [0]:
fake_pred